## DEMO DI UTILIZZO PROGETTO D'ESAME

Di seguito verrà simulato come un utente potrebbe interfacciarsi al database costruito per l'esame.

In [14]:
# Importo le librerie che  utilizzerò per la connessione al database
import mysql.connector
from mysql.connector import Error

Chidedo all'utente le credenziali per accedere al database e verifico se posso connettermi

In [15]:
done = False
usr = None
pwd = None

while done == False:
    usr = input("Inserire username: ")  # utente
    pwd = input("Inserire password: ")  # pwd_ESAM3
    try:
        connection = mysql.connector.connect(host='localhost', database = 'DRGbeer', user=usr, password=pwd)
        if connection.is_connected():
            print("Connessione al database riuscita")
            done = True
    except Error as e:
        print("Errore durante la connessione al database: ", e)
        done = False
    finally:
        if connection is not None and connection.is_connected():
            connection.close()
            print("Connessione chiusa")
    

Connessione al database riuscita
Connessione chiusa


Definisco una funzione per semplificare le chiamate alle stored procedure presenti nel database

In [16]:
def call_stored_procedure(sp, usr_data = None):
    # instauro la connessione
    connection = mysql.connector.connect(host='localhost', database='DRGbeer', user=usr, password=pwd)
    try:
        # Creazione del cursore ed esecuzione stored procedure
        cursor = connection.cursor()
        if usr_data is None:
            cursor.callproc(sp)
        else:
            cursor.callproc(sp, usr_data)
        connection.commit()
        
        # Recupero dei risultati
        for result in cursor.stored_results():
            res = result.fetchall()
            for row in res:
                print(row)

    except Error as e:
        print(f"Errore durante l'esecuzione della stored procedure: {e}")
    # Chiusura del cursore e della connessione
    if connection is not None and connection.is_connected():
        cursor.close()
        connection.close()
        print("Connessione al database chiusa")


Definisco un dizionario con le operazioni che si possono eseguire

In [17]:
# dizionario contenente i nomi delle operazioni eseguibili
procedure = {
    '1':'Inserimento giocatore',         # in: nome utente
    '2':'Inserimento partita',           # in: versione, giocatore, classe, incarico, anomalia, bioma, bb, yc, ms, sn
    '3':'Aggiornamento suggerimenti',    # batch
    '4':'Consiglio missione',            # in: genericità, materiale
    '5':'Aggiornamento Inventario',      # in: nome utente, bb, yc, ms, sn
    '6':'Aggiunta Licenza',              # in: nome utente, nome birra
    '7':'Licenze Sbloccabili',           # in: nome utente
    '8':'Licenze Bloccate',              # in: nome utente
    '9':'Termina il programma'
}

Definisco le operazioni presenti nel database:

### 1: Inserimento giocatore

In [18]:
def inserimento_giocatore():
    print('Inserimento giocatore')
    nome = input('Inserire il proprio nome: ')
    call_stored_procedure('InserimentoGiocatore',(nome,))

### 2: Inserimento partita

In [19]:
def inserimento_partita():
    print('Inserimento partita')
    ver = input('Inserire la versione di gioco: ')
    nome = input('Inserire nome utente: ')
    classe = input('Inserire la classe utilizzata: ')
    incarico = input('Inserire incarico completato: ')
    anomalia = input('Inserire anomalia se presente: ')
    bioma = input('Inserire bioma visitato: ')
    BB = int(input('Inserire BB ottenuti: '))
    YC = int(input('Inserire YC ottenuti: '))
    MS = int(input('Inserire MS ottenuti: '))
    SN = int(input('Inserire SN ottenuti: '))
    call_stored_procedure('InserimentoPartita',(ver,nome,classe,incarico,anomalia,bioma,BB,YC,MS,SN))

### 3: Aggiornamento suggerimenti

In [20]:
def aggiornamento_suggerimenti():
    print('Aggiornamento suggerimenti')
    call_stored_procedure('AggiornamentoSuggerimenti')

### 4: Consiglio missione

In [21]:
def consiglio_missione():
    print('Consiglio missione')
    gen = input('Inserire ampiezza di ricerca:\n  0 -> generico\n  .nomeutente -> specifico\n')
    mat = input('Inserire materiale richiesto (BB, YC, MS, Sn): ')
    call_stored_procedure('Consiglio', (gen, mat))

### 5: Aggiornamento inventario

In [22]:
def aggiornamento_inventario():
    print('Aggiornamento inventario')
    nome = input('Inserire nome utente: ')
    BB = int(input('Inserire quantità BB aggiornata: '))
    YC = int(input('Inserire quantità YC aggiornata: '))
    MS = int(input('Inserire quantità MS aggiornata: '))
    SN = int(input('Inserire quantità SN aggiornata: '))
    call_stored_procedure('AggiornamentoInventario', (nome,BB,YC,MS,SN))

### 6: Aggiunta licenza

In [23]:
def aggiunta_licenza():
    print('Aggiunta licenza')
    nome = input('Inserire nome utente: ')
    birra = input('Inserire nome birra sbloccata: ')
    call_stored_procedure('AggiuntaLicenza',(nome,birra))

### 7: Licenze sbloccabili

In [24]:
def licenze_sbloccabili():
    print('Licenze sbloccabili')
    nome = input('Inserire nome utente: ')
    call_stored_procedure('LicenzeSbloccabili',(nome,))

### 8: Licenze bloccate

In [25]:
def licenze_bloccate():
    print('Licenze bloccate')
    nome = input('Inserire nome utente: ')
    call_stored_procedure('LicenzeBloccate',(nome,))

Procedo ora a simulare l'utilizzo del database
Si consiglia di provare alcune delle seguenti operazioni:
- aggiornare l'inventario (5) di 'Giocatore1' dandogli una decina di risorse per tipo;
- controllare le licenze sbloccabili (7) da 'Giocatore1' e da 'Giocatore2';
- aggiungere la licenza (6) dal nome 'Blackout Stout' all'utente 'Giocatore1'; 
- visualizzare le licenze non sbloccate (8) da 'Giocatore3';
- chiedere consigli (4) per un materiale a scelta per un giocatore a scelta con genericità 0.

In [26]:
print('Operazioni disponibili: ')
for i in range(1,len(procedure)+1):
    print(str(i), ' -> ', procedure[str(i)])
end = False
while end == False:
    
    request = input('Selezionare una operazione: ')
    if request == '1':
        inserimento_giocatore()
    elif request == '2':
        inserimento_partita()
    elif request == '3':
        aggiornamento_suggerimenti()
    elif request == '4':
        consiglio_missione()
    elif request == '5':
        aggiornamento_inventario()
    elif request == '6':
        aggiunta_licenza()
    elif request == '7':
        licenze_sbloccabili()
    elif request == '8':
        licenze_bloccate()
    elif request == '9':
        print('chiusura programma in corso...')
        end = True
    else:
        print('opzione non valida, riprovare')
    

Operazioni disponibili: 
1  ->  Inserimento giocatore
2  ->  Inserimento partita
3  ->  Aggiornamento suggerimenti
4  ->  Consiglio missione
5  ->  Aggiornamento Inventario
6  ->  Aggiunta Licenza
7  ->  Licenze Sbloccabili
8  ->  Licenze Bloccate
9  ->  Termina il programma
Aggiornamento inventario
Connessione al database chiusa
Licenze sbloccabili
('Arkenstout', 'A blizzard freezes you and encases you in ice', 0, 0, 4, 0)
('Best Wurst Beer', 'No Effect', 0, 0, 0, 0)
('Blackout Stout', 'Immediately maxes your drunk level, causing you to pass out', 0, 0, 0, 3)
('Burning Love', 'You catch on fire', 0, 0, 6, 0)
('Dark Morkite', 'x1.44 Morkite mined', 2, 0, 0, 0)
('Gut Wrecker', 'You release a loud burp and shake the screen', 1, 0, 0, 4)
('Mactera Brew', 'You release a loud fart and create a green cloud around you', 1, 0, 0, 6)
('Pots O’ Gold', 'x4 Gold mined', 3, 0, 0, 0)
('Randoweisser', 'Your loadout and cosmetics will be randomized', 0, 0, 0, 10)
('Red Rock Blaster', 'x1.69 Max Health